# n-gram 언어 모델링

In [1]:
import nltk
from nltk.util import ngrams
from collections import Counter

In [2]:
text = "오늘은 날씨가 좋다. 오늘은 기분이 좋다. 오늘은 일이 많다. 오늘은 사람이 많다. 오늘은 날씨가 맑다."

In [3]:
tokens = nltk.word_tokenize(text)

In [ ]:
# 1-gram, 2-gram                                  # n-gram(연속된 n개 토큰 묶음) 만들 거임
unigram = tokens                                   # 1-gram은 그냥 토큰 리스트 자체
bigram = list(ngrams(tokens, 2))                   # 토큰을 2개씩 연속으로 묶은 (토큰1, 토큰2) bigram 리스트 생성

unigram_freq = Counter(unigram)                    # 각 토큰(1-gram)이 몇 번 나왔는지 빈도 세기
bigram_freq = Counter(bigram)                      # 각 bigram(2-gram 튜플)이 몇 번 나왔는지 빈도 세기


Counter({('.', '오늘은'): 4,
         ('오늘은', '날씨가'): 2,
         ('좋다', '.'): 2,
         ('많다', '.'): 2,
         ('날씨가', '좋다'): 1,
         ('오늘은', '기분이'): 1,
         ('기분이', '좋다'): 1,
         ('오늘은', '일이'): 1,
         ('일이', '많다'): 1,
         ('오늘은', '사람이'): 1,
         ('사람이', '많다'): 1,
         ('날씨가', '맑다'): 1,
         ('맑다', '.'): 1})

In [5]:
for (w1, w2), freq in bigram_freq.items():
    prob = freq / unigram_freq[w1]      # 조건부 확률 계산 (w1 뒤에 w2가 올 확률)
    print(f'P({w2}|{w1}) = {prob:.3f}')

P(날씨가|오늘은) = 0.400
P(좋다|날씨가) = 0.500
P(.|좋다) = 1.000
P(오늘은|.) = 0.800
P(기분이|오늘은) = 0.200
P(좋다|기분이) = 1.000
P(일이|오늘은) = 0.200
P(많다|일이) = 1.000
P(.|많다) = 1.000
P(사람이|오늘은) = 0.200
P(많다|사람이) = 1.000
P(맑다|날씨가) = 0.500
P(.|맑다) = 1.000


### Perplexity 계산

In [9]:
import math                                           # 로그, 거듭제곱 계산용 수학 라이브러리

# Perplexity의 평가 기준
# - 모델이 테스트 데이터에서 얼마나 적은 불확실성을 가지며 다음 단어를 잘 예측하는가
def compute_bigram_perplexity(test_text, unigram_freq, bigram_freq):  
    test_tokens = nltk.word_tokenize(test_text)       # 테스트 문장을 토큰(단어) 단위로 분리
    test_bigrams = list(ngrams(test_tokens, 2))       # 테스트 토큰으로부터 bigram 생성

    log_prob_sum = 0                                  # 로그 확률 누적 합 초기화
    N = len(test_bigrams)                             # 테스트 bigram 총 개수

    for bigram in test_bigrams:                       # 각 bigram에 대해 반복
        w1, w2 = bigram                               # bigram을 앞 단어(w1), 뒤 단어(w2)로 분리
        prob = bigram_freq.get(bigram, 0) / unigram_freq.get(w1, 1)  
                                                      # P(w2 | w1) = bigram 빈도 / unigram 빈도
        if prob == 0:                                 # 확률이 0이면
            prob = 1e-10                              # 로그 계산을 위해 아주 작은 값으로 대체
        log_prob_sum += math.log2(prob)               # 확률의 log2 값을 누적

    cross_entropy = -log_prob_sum / N                 # 평균 음의 로그 확률 = 크로스 엔트로피
    perplexity = math.pow(2, cross_entropy)           # perplexity = 2^(cross entropy)

    return perplexity                                 # 계산된 perplexity 반환


In [10]:
train_text = "자연어 처리는 재미있다. 자연어 처리는 어렵지만 도전하고 싶다. 오늘은 날씨가 좋다."  
                                                    # 학습용 말뭉치(훈련 텍스트)

train_tokens = nltk.word_tokenize(train_text)       # 문장을 단어(토큰) 단위로 분리

unigram = train_tokens                              # 1-gram: 각 단어 토큰 그대로 사용
bigrams = list(ngrams(train_tokens, 2))             # 2-gram: 연속된 두 단어씩 묶어 bigram 생성

unigram_freq = Counter(unigram)                     # 각 단어(unigram)의 등장 횟수 계산
bigrams_freq = Counter(bigrams)                     # 각 bigram(단어쌍)의 등장 횟수 계산


In [11]:
test_sentences = [                                 # 퍼플렉서티를 평가할 테스트 문장 목록
    "자연어 처리는 재미있다.",                      # 학습 데이터에 포함된 문장
    "자연어 처리는 어렵지만 도전하고 싶다.",        # 학습 데이터와 유사한 문장
    "오늘은 날씨가 좋다.",                          # 학습 데이터에 포함된 문장
    "기계 번역은 어렵다.",                          # 학습 데이터에 없는 문장
    "자연어 처리에 도전하고 싶다.",                 # 일부 단어/구조만 겹치는 문장
    "오늘 날씨가 흐리다."                           # 단어 순서·표현이 다른 문장
]

for sentence in test_sentences:                     # 각 테스트 문장에 대해 반복
    pp = compute_bigram_perplexity(
        sentence,                                   # 평가할 문장
        unigram_freq,                               # 학습된 unigram 빈도
        bigrams_freq                                # 학습된 bigram 빈도
    )
    print(sentence, "perplexity:", pp)               # 문장과 해당 perplexity 출력


자연어 처리는 재미있다. perplexity: 1.2599210498948732
자연어 처리는 어렵지만 도전하고 싶다. perplexity: 1.148698354997035
오늘은 날씨가 좋다. perplexity: 1.0
기계 번역은 어렵다. perplexity: 10000000000.000008
자연어 처리에 도전하고 싶다. perplexity: 100000.00000000003
오늘 날씨가 흐리다. perplexity: 10000000000.000008
